In [1]:
import sklearn as skl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [2]:

df = pd.read_csv('datasets/train_radiomics_hipocamp.csv')
df_control = pd.read_csv('datasets/train_radiomics_occipital_CONTROL.csv')

# Stats iniciais


In [3]:

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Columns: 2181 entries, ID to Transition
dtypes: float64(2014), int64(147), object(20)
memory usage: 5.1+ MB


In [4]:
# parecem o mesmo dataset???
df_control.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Columns: 2181 entries, ID to Transition
dtypes: float64(2014), int64(147), object(20)
memory usage: 5.1+ MB


In [5]:
print(df.equals(df_control))
# Check ignoring order
print(df.sort_index(axis=0).sort_index(axis=1).equals(df_control.sort_index(axis=0).sort_index(axis=1)))

# Parece que são diferentes apesar de terem as mesmas rows e collumns.

False
False


In [6]:
# Ainda não percebi a diferença entre dois e para que usar o de controlo.
print(df_control['Transition'].value_counts())
print(df['Transition'].value_counts())

Transition
CN-CN      96
MCI-MCI    71
MCI-AD     68
AD-AD      60
CN-MCI     10
Name: count, dtype: int64
Transition
CN-CN      96
MCI-MCI    71
MCI-AD     68
AD-AD      60
CN-MCI     10
Name: count, dtype: int64


# Null Values

In [7]:
# Nenhum valor null
df.isnull().sum().any()

False

# Dealing With features with only one value, No Variance

In [8]:
print(df.nunique().sort_values().head(20))

print("\nCount of features with only 1 value: %d" % sum(df.nunique() == 1) )

features_without_variance = df.columns[df.nunique()==1].to_list()

# df.drop( columns=features_without_variance,axis=1,inplace=True)

clean_df = df.drop( columns=features_without_variance,axis=1)

lbp-2D_glcm_SumAverage                              1
lbp-3D-m2_glrlm_LowGrayLevelRunEmphasis             1
lbp-2D_glszm_SizeZoneNonUniformityNormalized        1
lbp-2D_glszm_ZoneEntropy                            1
lbp-2D_ngtdm_Busyness                               1
lbp-3D-m2_glrlm_HighGrayLevelRunEmphasis            1
lbp-3D-m2_glrlm_GrayLevelVariance                   1
lbp-3D-m2_glrlm_GrayLevelNonUniformityNormalized    1
lbp-2D_ngtdm_Coarseness                             1
lbp-2D_ngtdm_Complexity                             1
lbp-2D_ngtdm_Contrast                               1
lbp-2D_ngtdm_Strength                               1
lbp-3D-m2_gldm_LowGrayLevelEmphasis                 1
lbp-2D_glszm_SizeZoneNonUniformity                  1
lbp-3D-m1_firstorder_Entropy                        1
lbp-3D-m2_gldm_GrayLevelVariance                    1
lbp-3D-m1_firstorder_Maximum                        1
lbp-3D-m1_firstorder_Minimum                        1
lbp-3D-m1_firstorder_Range  

# Object type features

In [9]:

columns_object = df.columns[df.dtypes == object]

In [10]:
df_object = df[columns_object]

df_object_converted = df_object.convert_dtypes()



In [11]:
columns_simple_object_all_distinct = ['ID', 
                                       # file paths
                                      'Image', 
                                      'Mask', 
                                       # hashes
    'diagnostics_Image-original_Hash', 'diagnostics_Mask-original_Hash']
                                      #
colums_simple_object_same_value = [ 
                                      'diagnostics_Versions_PyRadiomics',
        'diagnostics_Versions_Numpy', 'diagnostics_Versions_SimpleITK',
       'diagnostics_Versions_PyWavelet', 'diagnostics_Versions_Python']

colums_jsons_with_same_value = ['diagnostics_Configuration_Settings','diagnostics_Configuration_EnabledImageTypes']

tuples_with_same_value = ['diagnostics_Image-original_Spacing','diagnostics_Image-original_Size','diagnostics_Mask-original_Spacing','diagnostics_Mask-original_Size']


## Colunas Objeto sem valor de informação para os modelos

As colunas ['ID', 'Image', 'Mask', 'diagnostics_Image-original_Hash', 'diagnostics_Mask-original_Hash'] reprentam, IDs, diretorias de ficheiros e hashes. Cada uma contem valores completamente distintos e como é uma feature categórica não tem valor de informação.

As seguintes features são compostas por versões e têm sempre o mesmo valor, não tendo interesse. [ 
                                      'diagnostics_Versions_PyRadiomics',
        'diagnostics_Versions_Numpy', 'diagnostics_Versions_SimpleITK',
       'diagnostics_Versions_PyWavelet', 'diagnostics_Versions_Python']

As features de configuração, contêm um json, no entanto a configuração é sempre a mesma não havendo interesse em decompor o json em novas features ['diagnostics_Configuration_Settings','diagnostics_Configuration_EnabledImageTypes'].

As features de spacing e size contêm tuplos de 3 elementos, ['diagnostics_Image-original_Spacing','diagnostics_Image-original_Size','diagnostics_Mask-original_Spacing','diagnostics_Mask-original_Size']. Os tuplos são sempre iguais, não tem interesse.

## Colunas Objeto com algum potencial de informação

A feature diagnostics_Mask-original_BoundingBox é um tuplo com 6 dimensões com variação nos valores. Pode ser decomposta em 6 features, no entanto cada valor separado representa uma coordenada e não deve ter significado para a deteção. No entanto, algumas features derivadas podem ter interesse, como por exemplo o volume. 

As features diagnostics_Mask-original_CenterOfMassIndex e diagnostics_Mask-original_CenterOfMass são tuplos de 3 dimensões. Não tendo conhecimento do domínio, explorou se mais uma feature matemática, magnitude do vector em relação à origem.




# Conclusões/Notas

No module preprocessing.py foram adicionadas as transformações descritas.

Foi observado também que há features com um range de valores ora com floats muito pequenos ou bem grandes. Necessário scaling ou outro tipo de técnicas para alguns tipo de modelos ( com árvores não deve fazer diferença).

Há uma grande quantidade de features para um numero reduzido de amostras. Pode ser interessante explorar técnicas de feature selection ou outras.